In [1]:
from flu_model import FluModel, State
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ipywidgets for UI
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label

# Bokeh (visual) imports
from bokeh.io import output_notebook, show, export_svgs, push_notebook
from bokeh.models import ColumnDataSource, Legend, LegendItem, Range1d
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.layouts import row
from bokeh.models.annotations import Title

# silence missing renderers warning (empty plot must be set for updating)
from bokeh.core.validation.warnings import MISSING_RENDERERS
from bokeh.core.validation import silence
silence(MISSING_RENDERERS, True)

# set jupyter output
output_notebook()

# set color scheme
colors = ['#3EC9D6','#137F48', '#F9C149', '#aa110d']


Loading BokehJS ...

In [2]:
def plot_bokeh(p, model, N_steps, deceased, sampled):

    # get agent model results
    agent_state = model.datacollector.get_agent_vars_dataframe().drop(columns='p')
    
    # Change into more readable categorical data
    df = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size, fill_value=0)
    labels = ['Susceptible','Infected','Recovered']
    df.columns = labels[:len(df.columns)]
 
    # Reset indices and make ColumnDataSource object for plotting
    df.reset_index(inplace=True);
    df = pd.concat([df, pd.Series(deceased, name='Deceased')], axis=1)
    df = pd.concat([df, pd.Series(sampled, name='Sampled')], axis=1)

    # create ColumnDataSource object for Bokeh formatting
    source = ColumnDataSource(df)
    
    legend_items = []

    # Add each line to the figure and append to legend_items
    for i, c in enumerate(df.columns[1:]):
        
        if c == 'Sampled':
            p.line(x='Step', y=c, source=source, line_color=colors[1], 
                           line_dash='dotted', line_width=1.5, line_alpha=0.85)

        else:
            l = p.line(x='Step', y=c, source=source,
                        line_color=colors[i], line_width=3,
                        line_alpha=0.8)
            legend_items.append(LegendItem(label=c, renderers=[l]))

    # Set legend & title information
    p.add_layout(Legend(items=legend_items))
    p.legend.location = 'top_right'
    p.legend.background_fill_color = "#eaeaea"
    p.legend.background_fill_alpha = .8
    
    # disable axis names
    p.xaxis.major_label_text_color = None; p.yaxis.major_label_text_color = None 
    
    p.xaxis.axis_label = 'Time'
    p.toolbar.logo = None
    p.title.text_font_size = "20px"
    p.yaxis.axis_label = 'People'
    
    p.background_fill_color = "#cdcdcd"
    p.background_fill_alpha = 0.3
    
    p.toolbar.logo = None; p.toolbar_location = None

    return p

In [3]:
def plot_grid(p, model, color_map):
    
    p.renderers = []
    
    x_vals = []; y_vals = []; states = []
    
    # get occupants of each cell in the grid
    for cell in model.grid.coord_iter():
        agents, x, y = cell
        
        for agent in agents:
            x_vals.append(str(x))
            y_vals.append(str(y))
            states.append(str(agent.state))
                
    df = pd.DataFrame({'x': x_vals, 'y': y_vals, 'state': states})
    df = df.dropna()
    
    source2 = ColumnDataSource(df)

    # fill in rectangles at (x,y) location w/ color mapped from present state
    p.rect('x', 'y', width=1, height=1, source=source2,
        fill_color=color_map, fill_alpha=0.75, line_color='black')
    p.axis.axis_line_color = None
    p.grid.grid_line_color = None
    p.background_fill_color = '#161518'
    p.background_fill_alpha = 0.76
    
    # disable axis labels
    p.xaxis.axis_label = None; p.yaxis.axis_label = None
   
    # disable tickmarks
    p.xaxis.major_tick_line_color = None; p.xaxis.minor_tick_line_color = None
    p.yaxis.major_tick_line_color = None; p.yaxis.minor_tick_line_color = None  
    
    # disable axis names
    p.xaxis.major_label_text_color = None; p.yaxis.major_label_text_color = None 
    
    p.toolbar.logo = None; p.toolbar_location = None

    return p

In [4]:
def flu_simulation(pop, x_map, y_map, s, opts, N_steps, init_inf, output_file=None):

    # generate model 
    model = FluModel(pop, width=x_map, height=y_map, init_inf=init_inf)
    
    # initialization steps
    factors = ['State.SUSCEPTIBLE', 'State.INFECTED', 'State.RECOVERED']
    color_map = factor_cmap(field_name='state', palette=colors, factors=factors)
    deceased = []; sampled = []
    
    # create time + grid plots
    p1 = figure(**opts); p2 = figure(**opts)
    p1.output_backend = 'svg'; p2.output_backend = 'svg'
    
    # set title + range info
    t1 = Title(); t1.text = 'SIR Model of Pandemic Response'; p1.title = t1
    p2.x_range=Range1d(-1,x_map); p2.y_range=Range1d(-1,y_map)
    
    # show both plots and save handle for updating
    t = show(row(p1,p2), notebook_handle=True)

    for i in range(N_steps):
        
        # advance the flu model
        model.step()

        # append number of deceased from model to list
        deceased.append(model.deceased)

        # sample data for plotting 
        sampled.append(model.sample(s,i) * 100/s)

        # get repainted plots
        p1 = plot_bokeh(p1, model, N_steps, deceased, sampled)
        p2 = plot_grid(p2, model, color_map)
        
        # push changes to jupyter notebook
        push_notebook(handle=t)

    # OUTPUT .CSV OF RESULTS
    # get agent model results
    agent_state = model.datacollector.get_agent_vars_dataframe().drop(columns='p')

    # change into counts of categorical data, rename columns
    df = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size, fill_value=0)
    df.columns = ['Susceptible', 'Infected', 'Recovered']
    
    # add deceased and sampled (dashed-line) data to the dataframe
    df['Deceased'] = deceased
    df['Sampled'] = sampled
    
    # deal with output file provided
    if (output_file == ''):
        output_file = 'output.csv'
    elif (output_file[-4:].lower() != '.csv'):
        output_file = output_file + '.csv'
        
    df.to_csv(output_file)

In [5]:
# user Interface
button = widgets.Button(description='Run')

# gather selected values upon button push // submission
def gather(b=None):
    
    s = sample_slider.value
    biased = biased_check.value # not used yet
    
    pop = population_box.value
    infected = infected_slider.value
    x = map_size_x.value
    y = map_size_y.value
    nstep = steps_box.value
    output_file = output_fn.value
    
    # build opts for plotting
    opts = dict(plot_width=490, plot_height=490, 
                min_border=0, x_range=(0, nstep), toolbar_location=None)
    
    # begin mesa simulation
    flu_simulation(pop=pop, x_map=x, y_map=y, s=s, opts=opts, 
                   N_steps=nstep, init_inf=infected, output_file=output_file)

@button.on_click
def plot_on_click(b):
    gather()
    
button2 = widgets.Button(description='Fit to Population')

@button2.on_click
def calc_on_click(b):
    pop = population_box.value
    dim = int(np.ceil(np.sqrt(pop) * 1.4))
    
    map_size_x.value = dim
    map_size_y.value = dim

# set up first tab input fields
sample_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=100.0,
    step=0.1,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

biased_check = widgets.Checkbox()

# first tab for sampling info
tab1 = VBox(children=[HBox([Label('Percent Sampled'), sample_slider]),
                      HBox([Label('Biased Sampling'), biased_check])])

# set up second tab input fields
population_box = widgets.BoundedIntText(
    value=300,
    min=10,
    max=10000,
    step=1,
    orientation='horizontal'
)

steps_box = widgets.BoundedIntText(
    value=90,
    min=10,
    max=300,
    step=1,
    orientation='horizontal'
)

infected_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1)

map_size_x = widgets.BoundedIntText(
    value=25,
    max=1000,
    min=10,
    step=1,
    orientation='horizontal'
)

map_size_y = widgets.BoundedIntText(
    value=25,
    min=10,
    max=1000,
    step=1,
    orientation='horizontal'
)

# second tab for model details
tab2 = VBox(children=[HBox([Label('Population'), population_box]), HBox([Label('Number of Timesteps'), steps_box]),
                              HBox([Label('Initial Infected Percent'), infected_slider]),
                              HBox([Label('Map Size'), map_size_x, Label(' x '), map_size_y, button2])])

# set up third tab input fields
susceptible = widgets.Checkbox()
infected = widgets.Checkbox()
recovered = widgets.Checkbox()
deceased = widgets.Checkbox()
sampled = widgets.Checkbox()

output_fn = widgets.Text(
    placeholder='Enter filename . . . e.g. output.csv',
    disabled=False
)

# third tab for plot displays
tab3 = HBox([VBox([Label('Susceptible'), Label('Infected'), Label('Recovered'), Label('Deceased')]),
             VBox([susceptible, infected, recovered, deceased]), Label('Output'), output_fn])

# create complete user interface
tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Sampling')
tab.set_title(1, 'Model')
tab.set_title(2, 'Plotting')
VBox(children=[tab, button])
